## **Mount Gdrive - Optional for pdf file**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/ConceptsofBiology-WEB.pdf /content/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **RAG-Question Answer**

This code provides a Q A chat bot using Transformers - to answer questions from [this](https://openstax.org/details/books/concepts-biology) textbook.

From the assignment we have followed these:
1.   Here we will consider two chapters (3 & 4) from the entire textbook because it seems chapter 3 and 4 have better Q&A and use it as our knowledge base. So the chatbot will only provide answers related to these chapters only.
2. First we will pre-process (basic data cleaning) the entire corpus and then split it into multiple chunks.
3.   We used *'Retrieve & Re-Rank'* to find the relevant documents for a query as it gives much better result.
4.   Then we used *Open sources LLM* model from huggingface to find the answer to the query.
5. We have also created a chatbot UI using [Gradio](https://github.com/gradio-app/gradio)

**References** :

*   We used the library [sentence-transformers](https://github.com/UKPLab/sentence-transformers) for this project. This library provides an easy method to compute dense vector representations for sentences, paragraphs and also has a lots of pretrained models.

* Huggingface 🤗 model hub for pretrained models.

### Installing Required Libraries

In [ ]:
%%capture

!pip install -r requirements.txt

In [1]:
# running on gpu
import torch

if torch.cuda.is_available():
    device = torch.device('cuda:0')

torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)

In [2]:
# Importing libraries

import os, re
import fitz
import uuid
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)

from tqdm import tqdm
import torch
import chromadb
import transformers
import pandas as pd
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from sentence_transformers.cross_encoder import CrossEncoder

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import warnings
warnings.filterwarnings("ignore")

### Data Pre-Processing Pipeline

In [4]:
#### place the pdf document downloded from the given link in the current folder ####

try:
    doc = fitz.open("/content/drive/MyDrive/ConceptsofBiology-WEB.pdf") # open the document
    all_text= []

    for page in tqdm(doc, desc="Extracting Text", unit="page"):
        text = page.get_text()
        all_text.append(text)

    print("Text extraction successful.")
except Exception as e:
    print(f"Error occurred: {str(e)}")

Extracting Text: 100%|██████████| 615/615 [00:01<00:00, 473.14page/s]

Text extraction successful.


In [6]:
# we are pre processing the entire text for Chapter 3 & 4 (Page 55 to 113)

ignore_sentences = ["Introduction", "Introduction to Biology", ""]

def remove_ignored_sentences(sentences, ignore_sentences):
    ignore_set = set(sentence.lower() for sentence in ignore_sentences)

    # Filter out sentences that are in the ignore list
    filtered_sentences = [sentence for sentence in sentences if sentence.lower() not in ignore_set]
    # Further we can use NLTK to filter stop words and etc..

    return filtered_sentences





def split_sentences(text):
    # Split on a period that is not preceded by a digit and not followed by a digit
    # or at the beginning or end of the string.
    sentences = re.split(r'(?<!\d)\.(?!\d)', text)
    # Remove empty strings from the list and strip whitespace
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences





# list containing the lines of text
chapter_3_4 =[]
for text in all_text[55:113]:

    sentences= text.split('\n')

    filtered_sentences = remove_ignored_sentences(sentences, ignore_sentences)
    sample_text= " ".join(filtered_sentences)
    chapter_3_4.extend(split_sentences(sample_text))

In [7]:
# Function to count words in a sentence

chunk_size= 500       # Chunk size
n_overlap= 25         # Max number of common words between consecutive chunks

def split_into_chunks(list_of_sentences, chunk_size, n_overlap):

    """
    Split the list of Random Chunks into uniform chunksize with word overlaps

      Args:
          list_of_sentences (list(str)): The first number.
          chunk_size (int): The chunk size.
          n_overlap (int): Number of common words between two chunks.

      Returns:
          list: List of chunks .

    """

    def count_words(sentence):
        return len(sentence.split())

    chunks = []
    current_chunk = []
    word_count = 0

    for sentence in list_of_sentences:
        sentence_word_count = count_words(sentence)

        # Check if adding the current sentence would exceed the chunk size
        if word_count + sentence_word_count > chunk_size and current_chunk:
            chunks.append(current_chunk)
            # Start new chunk with overlap if needed
            overlap_words = 0
            overlap_sentences = []
            if n_overlap > 0:
                for sent in reversed(current_chunk):
                    overlap_words += count_words(sent)
                    overlap_sentences.insert(0, sent)
                    if overlap_words >= n_overlap:
                        break
            current_chunk = overlap_sentences.copy()
            word_count = sum(count_words(sent) for sent in current_chunk)

        current_chunk.append(sentence)
        word_count += sentence_word_count

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk)

    chunks= [" ".join(chunk) for chunk in chunks]

    return chunks



# Ensure each sentence ends with a period
chapter_3_4 = [sentence if sentence.endswith('.') else sentence + '.' for sentence in chapter_3_4]

chunks = split_into_chunks(chapter_3_4, chunk_size, n_overlap)

In [8]:
# creating unique ids from each chunks
data = [{"chunk": chunk, "chunk_id": f"cid_{str(uuid.uuid4())}"} for chunk in chunks]


df = pd.DataFrame(data)
df

,chunk,chunk_id
0,"biological macromolecule chitin, which is a ni...",cid_9a77e0af-ea63-46c2-afa0-081e7a69eeb7
1,Fatty acids have a long chain of hydrocarbons ...,cid_2a8b9ef5-5a39-4cd5-a123-e4cc022ae62f
2,"Simply speaking, hydrogen gas is bubbled throu...",cid_11a81dd9-3613-4723-bb99-c4b448eac757
3,"The fatty acids of phospholipids face inside, ...",cid_e17daeef-bc3a-459d-be8c-eaddaa481e29
4,"For example, insulin is a protein hormone that...",cid_e3fa5b2d-bcd0-43bf-97a5-582c2259fc6b
5,Protein sequencing has shown that there is a c...,cid_4c67ae88-d395-4d66-8731-19500aa177fc
6,Both structures are held in shape by hydrogen ...,cid_817e2a87-4ae9-48db-b0fc-2b90aa9d2b5d
7,Nucleic Acids Nucleic acids are key macromolec...,cid_133c1f9e-fef3-48ba-8fda-d9c01b35a884
8,The bases pair in such a way that the distance...,cid_24ea8388-b1ca-48fb-83ff-c8dff96552a9
9,Key Terms acid a substance that donates hydrog...,cid_b6cdf2db-45de-4ae3-bf63-3c2257baecec


### Creating Embeddings for Text Chunks and Storing in VectorDB

In [9]:
# Initiating BGE Embedding Model

embed_model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {"normalize_embeddings": True}
embed_model = HuggingFaceBgeEmbeddings(
    model_name=embed_model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [10]:
# Storing in VectorDB

def store_in_db():

  """
  Returns a chromadb object with embeddings for all the text chunks.
  """
  try:
      client = chromadb.Client()
      test_collection= client.create_collection("biology-text-qa", metadata={"hnsw:space": "cosine"})
      documents, embeddings, ids= [], [], []
      for index, row in df.iterrows():

          documents.append(row['chunk'])

          embedding = embed_model.embed_query(row['chunk'])
          embeddings.append(embedding)

          ids.append(row['chunk_id'])


      test_collection.add(
          documents= documents,
          embeddings= embeddings,
          ids= ids
      )

  except Exception as e:
      print(e)
      test_collection= client.get_collection("biology-text-qa")

  return test_collection


test_collection= store_in_db()

### Creating Method for Retrieval

In [11]:
# Initiating Cross-Encoder Model for reranking top documents

cross_encoder_model = CrossEncoder('sentence-transformers/all-mpnet-base-v2')

Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Method to find top Documents

def find_top_docs(question, db_collection, cross_encoder_model, embed_model, top_k=3):

    """
    Find the top-k documents matching the query and also send the most suitable
    document after re-ranking depending on the input question.

      Args:
          question (str): The Question from user.
          db_collection (collection): object with embeddings for all the text chunks.
          cross_encoder_model (CrossEncoder): Model for reranking top documents found in initial search result.
          embed_model(HuggingFaceBgeEmbeddings): Model for embedding all text chunks
          top-k (int): Number of chunks most related to the query.

      Returns:
          str: most suitable document depending on the input question.

    """

    qembed= embed_model.embed_query(question)
    result= db_collection.query(
        query_embeddings= qembed,
        n_results= top_k
    )


    top_docs_list =result['documents'][0]

    cross_input = [[question, docs] for docs in top_docs_list]
    cross_scores = list(cross_encoder_model.predict(cross_input))

    # Combine the scores and docs and sort them
    combined = sorted(zip(cross_scores, top_docs_list), reverse=True)

    # Extract the sorted docs
    sorted_docs = [docs for _, docs in combined]

    return sorted_docs[0], combined

In [13]:
# Initializing Model and text generation pipeline for question answering

model_id = "HuggingFaceH4/zephyr-7b-beta"


qa_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda:0",
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,

)

qa_tokenizer = AutoTokenizer.from_pretrained(model_id)


max_num_of_words= 500  # Keeping max word length to 500 words
qa_pipeline =pipeline("text-generation",
                model=qa_model,
                tokenizer=qa_tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="cuda:0",
                max_new_tokens=max_num_of_words,
                min_new_tokens=-1,
                )

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### Method to find the Answer for the Input Question

In [14]:
def generate_answer(question, context,  pipeline, tokenizer):

    """
    Returns answer for the input question using the provided context.

      Args:
          question (str): The Question from user.
          context (str): The chunk most suitable to the question.
          pipeline (TextGenerationPipeline ): Huggingface text generation pipeline.
          tokenizer(LlamaTokenizerFast): Tokenizer for the QA-Model for tokenizing input prompt.

      Returns:
          str: Answer to the question from the provided context .

    """

    # Creating Prompt template
    message = [
        {"role": "system", "content": f"""You are a helpful assistant chatbot. You will be given a context and a question. You will understand the provided context
        below and will answer the asked question in a concise and crisp manner."""},

        {"role": "user", "content": f""" Understand the given context and
        try to answer within {max_num_of_words} words.
        also try to anwer in short do not make bigger sentense if not neccessary.
        Do not use your previous knowledge for the answer. If you are
        not able to find the answer in the given context, just say that you don't know, don't try to make up an answer.
        Make sure that you complete the answer within {max_num_of_words} words.

        Question: {question}

        Context: {context}"""}
    ]
    prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

    sequences = pipeline(
            prompt,
            do_sample=True,
            top_k=50,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=max_num_of_words,
            temperature=0.1,
            top_p=0.95,
            return_full_text=False,
        )

    answer = sequences[0]['generated_text']

    return answer, prompt

In [16]:
def qa_engine(question, history):

    try:
      # Find most suitable text chunk
      top_doc, _= find_top_docs(question, test_collection, cross_encoder_model, embed_model, top_k=5)

      # Find answer for the input question
      results, _ = generate_answer(question, top_doc, qa_pipeline, qa_tokenizer)

      return results

    except:
      return "Sorry!! Unable to Answer."

In [19]:
# Setting Up Gradio

import gradio as gr



gr.ChatInterface(
    qa_engine,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Question And Answer ChatBot for Concepts on Biology",
    description="Ask me questions regarding Biology chapter 3 and 4",
    theme="soft",
    examples=["What is Cell Function ?", "What is glucose ?"],
    retry_btn="TryAgain",
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2ce955bb908e0002b8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2ce955bb908e0002b8.gradio.live


## **Further Improvements**

areas to improve are :

1.   To improve the retrieval process, we could have extracted metadata from chunks of text, and used it for keyword retrieval like BM25 along with dense vector embeddings.
2. To improve the inference time, we could have used ctransformer or vLLM (paged attention) for the LLM model.
3. Experiment with many new models like "openchat 3.5" etc for better answers.
